In [1]:
%%configure -f
{"driverMemory": "4g", "executorMemory":"8960m"}

In [2]:
# Packages to download the corpus and pandas
import urllib.request
#import pandas as pd
# This block is for sparknlp stuff
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
# This block is for pyspark stuff
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import BucketedRandomProjectionLSH, BucketedRandomProjectionLSHModel

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1645496349994_0010,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = sparknlp.start()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
urls = spark.read.json("s3://your-bucket/datasets/aws-blogs/json/list.json")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
urls.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- contentType: string (nullable = true)
 |-- contributors: string (nullable = true)
 |-- createdDate: string (nullable = true)
 |-- displayDate: string (nullable = true)
 |-- featuredImageUrl: string (nullable = true)
 |-- link: string (nullable = true)
 |-- modifiedDate: string (nullable = true)
 |-- postExcerpt: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- title: string (nullable = true)

In [17]:
urls.select(F.col("postExcerpt"),F.col("title"),F.col("link")).show(1,1000)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                                                                                                                                                                               postEx

In [23]:
medlineplusDF = urls.withColumn(
    "text",
    F.concat(
        F.col("title"), 
        F.lit(" "), 
        F.col("postExcerpt")
    )
).select("title", F.col("link").alias("url"), "text")

medlineplusDF.show(1,1000)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                     title|                                                                                                                                                      url|                                     

In [24]:
docass = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector_dl = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

emb_use = UniversalSentenceEncoder.pretrained()\
    .setInputCols("sentence")\
    .setOutputCol("use_embeddings")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]

In [25]:
pipeline_use = Pipeline(
    stages=[
        docass, 
        sentence_detector_dl, 
        emb_use
        ]
    )

# Fit (or prepare) the pipeline into a model
model_use = pipeline_use.fit(medlineplusDF)

# Apply the model to our medlineplusDF content and save in a new dataframe
medlineplusSentencesDF_init = model_use.transform(medlineplusDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
medlineplusSentencesDF = medlineplusSentencesDF_init.select(
  F.col("title"),
  F.col("url"),
  F.arrays_zip(
    F.col("sentence.result").alias("sentence"),
    F.col("sentence.begin").alias("begin"),
    F.col("sentence.end").alias("end"),
    F.col("use_embeddings.embeddings")
  ).alias("zip")
).select(
  F.col("title"),
  F.col("url"),
  F.explode(F.col("zip")).alias("zip")
).select(
  F.col("title"),
  F.col("url"),
  F.col("zip")['0'].alias("sentence"),
  F.col("zip")['1'].alias("begin"),
  F.col("zip")['2'].alias("end"),
  F.col("zip")['3'].alias("embeddings")
).repartition(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
def avg_vectors(bert_vectors):
  length = len(bert_vectors[0]["embeddings"])
  avg_vec = [0] * length
  for vec in bert_vectors:
    for i, x in enumerate(vec["embeddings"]):
      avg_vec[i] += x
    avg_vec[i] = avg_vec[i] / length
  return avg_vec

import pyspark.sql.types as T

avg_vectors_udf = F.udf(avg_vectors, T.ArrayType(T.DoubleType()))

myudf = F.udf(lambda vs: Vectors.dense(vs), VectorUDT())

df_doc_vec = medlineplusSentencesDF.withColumn("doc_vector", avg_vectors_udf(F.col("embeddings")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
medlineplusSentencesDF_dv = df_doc_vec.select(
  "title", 
  "url", 
  "sentence", 
  "begin", 
  "end", 
  myudf("embeddings").alias("embeddings")
).repartition(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
def get_key(query, model):
  queryDF = spark.createDataFrame([[query]]).toDF("text")
  queryDF = model.transform(queryDF)
  queryDF = queryDF.select(
    F.explode(
      F.arrays_zip(
        F.col("sentence.result"),
        F.col("use_embeddings.embeddings")
      )
    ).alias("zip")
  ).select(
    F.col("zip")['0'].alias("sentence"),
    myudf(F.col("zip")['1']).alias("embeddings")
  )  

  key = queryDF.select("embeddings").take(1)[0].embeddings
  return key

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
get_key("How to stream from Hbase?", model_use)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DenseVector([0.0633, -0.0581, -0.0637, 0.0431, 0.0389, 0.0371, 0.027, -0.0193, -0.0623, -0.0135, 0.052, -0.0126, 0.0387, 0.0753, 0.0145, -0.0005, 0.0031, 0.0337, 0.0652, -0.041, -0.0444, -0.0724, -0.0748, -0.033, 0.0276, -0.0536, -0.0605, -0.0613, -0.0618, -0.0119, 0.0232, -0.0297, -0.0379, 0.0469, -0.016, 0.0386, 0.054, 0.0021, 0.0303, 0.0046, -0.0024, 0.0188, -0.0132, 0.0758, -0.0707, 0.0633, 0.0572, -0.0371, 0.018, 0.0048, 0.0592, 0.0115, 0.0762, -0.0594, 0.0389, 0.0451, 0.0577, -0.0369, 0.0364, -0.0261, -0.0349, 0.0166, 0.0583, 0.0052, -0.0022, -0.0247, 0.0462, -0.0701, 0.002, 0.0069, -0.049, -0.0553, 0.013, 0.0632, -0.0392, -0.0687, 0.029, -0.0453, -0.0143, -0.0474, -0.0227, 0.0477, 0.0401, 0.0076, 0.0035, 0.0093, -0.0423, -0.0477, -0.0513, -0.062, 0.0397, -0.002, -0.0058, -0.0326, 0.0015, -0.0774, 0.0036, -0.0197, -0.0187, -0.0491, -0.0698, -0.0242, -0.0085, 0.0033, 0.0106, 0.0733, 0.0759, 0.0114, -0.0074, 0.0493, -0.02, 0.022, 0.0071, 0.0198, -0.0278, 0.0418, 0.0595, -0.0517, -0

In [31]:
brp = BucketedRandomProjectionLSH(
 inputCol="embeddings",
 outputCol="hashes",
 bucketLength=10,
 numHashTables=5
)

# Fit the LSH model for our content
brp_model = brp.fit(medlineplusSentencesDF_dv)

# Apply the model to our content and get the new reduced
# representation
hashesDF = brp_model.transform(medlineplusSentencesDF_dv)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
def find_close_sentences(query, emb_model, brp_model, hashesDF, k):
  key = get_key(query, emb_model)
  resultsDF = brp_model.approxNearestNeighbors(hashesDF, key, k)
  return resultsDF

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
find_close_sentences("Optimize spark streaming?", model_use, brp_model, hashesDF, 5).select(F.col("title"),F.col("distCol"),F.col("sentence")).show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…